# Exploratory Data Analysis - Tracked with Weights and Biases

This notebook carries out basic Exploratory Data Analysis (EDA) of the [Dataset of songs in Spotify](https://www.kaggle.com/datasets/mrmorj/dataset-of-songs-in-spotify); its **goal is to showcase how to track notebook executions with Weight and Biases**. It assumes:

- The dataset has been uploaded to W&B using the CLI; see the `README.md` file.
- You have started this notebook via `mlflow run .` to correctly create the required conda environemnt specified in `conda.yaml`; see the `README.md` file.

All in all, the follosing tasks are carried out here:

- Start a new run `run = wandb.init(project="music_genre_classification", save_code=True)`; `save_code=True` makes possible to track the code execution.
- Download the dataset artifact and explore it briefly.
- Perform a simple EDA:
  - Run `pandas_profiling.ProfileReport()`.
  - Drop duplicates.
  - Impute missing song and tile values with `''`.
  - Create new text field which is the concatenation of the title and the song name.
- Finish the run: `run.finish()`.

To have a look at the data modeling, see th eother notebooks -- which are not tracked!

In [1]:
import wandb

In [2]:
import pandas_profiling
import pandas as pd
import seaborn as sns

In [3]:
# Create a run for our project; name automatically generated
run = wandb.init(
  project="music_genre_classification",
  save_code=True
)

wandb: Currently logged in as: datamix-ai (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
# Open the artifact: the name is not the filename,
# but the name we used when registering it
# To download the file we need to call .file()
artifact = run.use_artifact("music_genre_classification/genres_mod.parquet:latest")
df = pd.read_parquet(artifact.file())

In [5]:
df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,duration_ms,time_signature,genre,song_name,title
0,0.831,0.814,2,-7.364,1,0.4200,0.0598,0.013400,0.0556,0.3890,156.985,audio_features,124539,4,Dark Trap,Mercury: Retrograde,None
1,0.719,0.493,8,-7.230,1,0.0794,0.4010,0.000000,0.1180,0.1240,115.080,audio_features,224427,4,Dark Trap,Pathology,None
2,0.850,0.893,5,NaN,1,0.0623,0.0138,0.000004,0.3720,0.0391,218.050,audio_features,98821,4,Dark Trap,Symbiote,None
3,0.476,0.781,0,-4.710,1,0.1030,0.0237,0.000000,0.1140,0.1750,186.948,audio_features,123661,3,Dark Trap,ProductOfDrugs (Prod. The Virus and Antidote),None
4,0.798,0.624,2,-7.668,1,0.2930,0.2170,0.000000,0.1660,0.5910,147.988,audio_features,123298,4,Dark Trap,Venom,None


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42896 entries, 0 to 42895
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      42896 non-null  float64
 1   energy            42896 non-null  float64
 2   key               42896 non-null  int64  
 3   loudness          33726 non-null  float64
 4   mode              42896 non-null  int64  
 5   speechiness       42896 non-null  float64
 6   acousticness      42896 non-null  float64
 7   instrumentalness  42896 non-null  float64
 8   liveness          42896 non-null  float64
 9   valence           42896 non-null  float64
 10  tempo             42896 non-null  float64
 11  type              42896 non-null  object 
 12  duration_ms       42896 non-null  int64  
 13  time_signature    42896 non-null  int64  
 14  genre             42896 non-null  object 
 15  song_name         21811 non-null  object 
 16  title             21079 non-null  object

In [7]:
# Note: Jupyter Lab has sometimes issues; use Jupyter Notebook if you come up with them
profile = pandas_profiling.ProfileReport(df, title="Pandas Profiling Report", explorative=True)
profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
# Drop duplicates
df = df.drop_duplicates().reset_index(drop=True)

In [9]:
# New feature
# This feature will have to go to the feature store.
# If you do not have a feature store,
# then you should not compute it here as part of the preprocessing step.
# Instead, you should compute it within the inference pipeline.
df['title'].fillna(value='', inplace=True)
df['song_name'].fillna(value='', inplace=True)
df['text_feature'] = df['title'] + ' ' + df['song_name']

In [10]:
# Finish run to upload the results.
# Close the notebook and stop the jupyter server
# by clicking on Quit in the main Jupyter page (upper right).
# NOTE: DO NOT use Crtl+C to shutdown Jupyter.
# That would also kill the mlflow job.
run.finish()

In [11]:
# Go to W&B web interface: select run.
# You will see an option {} in the left panel.
# Click on it to see the uploaded Jupyter notebook.